In [1]:
import pandas as pd
from io import StringIO

from selenium import webdriver

import time
from datetime import datetime

driver=webdriver.Chrome()

In [2]:
data_contratos = '12/07/2023'

In [3]:
url = f'''
https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/SistemaPregao1.asp?
pagetype=pop&caminho=Resumo%20Estat%EDstico%20-%20Sistema%20Preg%E3o&Data={data_contratos}&Mercadoria=WDO
'''
driver.get(url)

local_tabela = '''
/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[3]/table
'''
local_indice = '''
/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[1]/table
'''

elemento = driver.find_element("xpath", local_tabela)
elemento_indice = driver.find_element("xpath", local_indice)

html_tabela = elemento.get_attribute('outerHTML')
html_indice = elemento_indice.get_attribute('outerHTML')

tabela = pd.read_html(StringIO(html_tabela))[0]
indice = pd.read_html(StringIO(html_indice))[0]

In [4]:
indice

,0
0,VENCTO
1,Q23
2,U23
3,V23
4,X23
5,Z23
6,F24
7,G24
8,H24
9,J24


In [5]:
tabela.columns = tabela.loc[0]
tabela = tabela['AJUSTE']
tabela = tabela.drop(0, axis = 0)
indice.columns = indice.loc[0]
indice_dolar = indice['VENCTO']
indice = indice.drop(0, axis = 0)
tabela.index = indice['VENCTO']
tabela = tabela[tabela != "0"]

print(tabela)

VENCTO
Q23    4.835,8110
U23    4.864,9520
V23    4.886,5200
X23    4.909,0650
Z23    4.926,8540
F24    4.942,8690
G24    4.963,4900
H24    4.976,4480
J24    4.991,3670
K24    5.007,6890
V24    5.081,8000
F25    5.123,8760
Name: AJUSTE, dtype: object


In [6]:
legenda = pd.Series(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                    index = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z'])

lista_datas = []

for indice in tabela.index:

    letra = indice[0]
    ano = indice[1:3]

    mes = legenda[letra]

    data = f"{mes}-{ano}"

    data = datetime.strptime(data, "%b-%y")

    lista_datas.append(data)


tabela.index = lista_datas
tabela

2023-08-01    4.835,8110
2023-09-01    4.864,9520
2023-10-01    4.886,5200
2023-11-01    4.909,0650
2023-12-01    4.926,8540
2024-01-01    4.942,8690
2024-02-01    4.963,4900
2024-03-01    4.976,4480
2024-04-01    4.991,3670
2024-05-01    5.007,6890
2024-10-01    5.081,8000
2025-01-01    5.123,8760
Name: AJUSTE, dtype: object

In [8]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"

fig= go.Figure()

fig.add_trace(go.Scatter(x=tabela.index, y=tabela.values, name=f"Dolar futuro",
                        line=dict(color='royalblue', width=2), mode='lines+markers'))

fig.update_layout(yaxis=dict(tickformat=".1%", tickfont=dict(color="black")),
                            xaxis=dict(tickfont=dict(color="black")))

fig.show()

In [7]:
driver.quit()